# Almgren-Chriss Framework Agent : 
# Quadratic Execution Costs & Constant Volume
## Based on the book : The Financial Mathematics of Market Liquidity: From Optimal Execution to Market Making 
#### This notebook shows how to fit the parameters of the classical sine equation of optimal liquidation of a single stock in the Almgren & Chriss Framework with quadratic execution costs :
#### It also shows how to backtest the fitted trading curves
# Imports

In [ ]:
# Imports
from IPython.display import display, HTML
import os
import pandas as pd, datetime as dt, numpy as np, matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset
import sys
import datetime as dt
from apscheduler.schedulers.background import BackgroundScheduler
from collections import deque # a faster insert/pop queue
from decimal import Decimal

# limit logging of scheduler
import logging
logging.basicConfig()
logging.getLogger('apscheduler').setLevel(logging.CRITICAL)

# Display options
thisnotebooksys = sys.stdout
pd.set_option('display.width', 1000)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('mode.chained_assignment', None)

In [ ]:
import mimicLOB as mlob

# LOB creation

In [ ]:
# b_tape = True means the LOB is taping all transactions
LOB = mlob.OrderBook(tick_size  = 0.5, 
                     b_tape     = True,
                     b_tape_LOB = True,
                     verbose    = False)

## Create the QRM Agent

In [ ]:
qrm_config = {'orderbook'     : LOB,
              'id'            : 'market',
              'b_record'      : True,
              'lambdas_plus'  : 1/pd.read_pickle(r'..\data\Lambda_plus.pkl'),
              'lambdas_minus' : 1/pd.read_pickle(r'..\data\Lambda_minus.pkl'),
              'event_sizes'   : pd.read_pickle(r'..\data\event_sizes.pkl'),
              'S0'            : 4400,
              'theta'         : 0.2, # Probability that the reference price (around which the lob dynamics are constructed) changes when the mid price changes (when a first limit is depleted)
              'theta_reinit'  : 0.7, # Probability that the whole LOB dynamics to be redrawn from its invariant distribution when the ref price changes (this transition is done smoothly : no huge jump in quantities between events)
              'MOPart'        : 0.05, # % of liquidity consuming events that are market orders (5% is a historical realized value for CF1)
              'verbose'       : False}

qrm = mlob.QRM(**qrm_config)

# Launch Simulation - Basic
#### Here we simulate a week of trading to fit our optimal trading strategy parameters 
#### The simulation lasts approx 4min

In [ ]:
%%time 

Hours_Of_Trading = 5*7 # approx 7 sec of simulation per hour 

for i in range(Hours_Of_Trading * 60**2):
    qrm.sendOrders()

#### Get Simulated Prices Evolution

In [ ]:
# Takes time
LOBtape                 = qrm.getLOBTape()
TransactionTape         = qrm.getTransactionTape()
histoPrices             = qrm.getPriceTape().astype(float)

LOBtape['TIME']         = pd.to_datetime(LOBtape['TIME'], unit='s')
TransactionTape['time'] = pd.to_datetime(TransactionTape['time'], unit='s')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(25, 8))

# bid & ask
LOBtape['TIME'] = pd.to_datetime(LOBtape['TIME'], unit='s')
bidask          = LOBtape.set_index('TIME').resample('s').last()[['BID0', 'ASK0']].dropna().astype(float)

# Mid Price Plot
bidask[['BID0', 'ASK0']].astype(float).plot(ax = ax1); ax1.set_title('bid & ask')

# Transaction price
histoPrices.plot(ax = ax2)

# Hourly Traded Quantity
TransactionTape.set_index('time').traded_quantity.rolling('1h').sum().plot(ax=ax3); ax3.set_title('Rolling 1 hour traded volume')

# Hourly Volatility (in Dollars)
midPrice = ((LOBtape.set_index('TIME').BID0 + LOBtape.set_index('TIME').ASK0)/2).iloc[2:]
midPrice.sort_index().diff(1).rolling('1h').std().plot(ax=ax4); ax4.set_title('Rolling intraday volatility - 1h')

plt.tight_layout()

#### Some statistics

In [ ]:
# OHLC
display(f'open  : {histoPrices.iloc[0,0]}')
display(f'high  : {histoPrices.max()[0]}')
display(f'low   : {histoPrices.min()[0]}')
display(f'close : {histoPrices.iloc[-1, 0]}')

# Volatility
display('annualized volatility : ', 100 * (6 * 60 * 7 * 252)**0.5 * TransactionTape.set_index('time').traded_price.astype(float).resample('10s').last().pct_change(1).std())

# Hourly Volatility 
display('hourly volatility in Dollars: ', TransactionTape.set_index('time').traded_price.resample('1h').last().diff(1).astype(float).std())

#Hourly Volume
display('hourly average volume in nb of transactions: ', TransactionTape.set_index('time').traded_quantity.rolling('1h').sum().mean())

# Optimal Trading 
## Parameters
#### Liquidity parameter &eta;
Quadratic trading cost function : L(&rho;) = &eta;&rho;2, <br>
&eta; is a scaling factor for the execution costs paid by the trader. <br>
The larger &eta; the more the trader pays to buy/sell shares.
#### Liquidity parameter V : Constant available volume
Similarly, the value of the market volume V is a scaling factor, because the execution costs depend on the <br>
participation rate. Hence, a small value of V has the same effect as a large value of &eta;.
#### Volatility parameters &sigma;
The volatility parameter &sigma; measures the importance of price risk. <br>
Therefore, the larger &sigma;, the faster the execution to reduce the exposure to price risk.
#### Risk Aversion Parameter &gamma;
The risk aversion parameter sets the balance between execution costs on the one hand, and price risk on the other hand. <br>
The larger the parameter, the more the trader is sensitive to price risk. <br>
Therefore, high &gamma; means fast execution to reduce the exposure to price risk. <br> 
When &gamma; is really small, we find that the optimal trading curve q* gets close to a straight-line strategy.<br>
In contrast, the liquidation is very fast at the beginning, when &gamma; is large.

In [ ]:
def alpha(gamma, sigma, V, nu, dt):
    cosh_alpha_dt = 1 + (gamma * V * (dt*sigma)**2) / (4 * nu)
    return (1/dt) * np.arccosh(cosh_alpha_dt)

def q_star(x, q0, T, gamma, sigma, V, nu, dt):
    alpha_ = alpha(gamma, sigma, V, nu, dt)
    return q0 * np.sinh(alpha_*(T - x))/np.sinh(alpha_*T)

In [ ]:
S0        = 4400
sigma     = TransactionTape.set_index('time').traded_price.resample('1h').last().diff(1).astype(float).std()
V         = TransactionTape.set_index('time').traded_quantity.rolling('1h').sum().mean()
nu        = 0.1 # dollar per contract
gamma     = 10**-5 #per dollar
timesteps = np.arange(0, 1, 1/60**2) 
q0        = 300 #int(0.1 * V) # nb of shares to liquidate : 10% of total traded volume
T         = 1
dt        = 1/60**2

## Some Trading Curves

In [ ]:
pd.DataFrame({'Gamma 10-4' : [q_star(x, q0, T, 10**-4, sigma, V, nu, dt) for x in timesteps],
              'Gamma 5*10-5' : [q_star(x, q0, T, 5*10**-5, sigma, V, nu, dt) for x in timesteps],
              'Gamma 10-5' : [q_star(x, q0, T, 10**-5, sigma, V, nu, dt) for x in timesteps]}).plot(figsize=(15, 7))

## Backtest of a trading Curve

### Launch the market simulation

In [ ]:
LOB = mlob.OrderBook(tick_size  = 0.5, 
                     b_tape     = True,
                     b_tape_LOB = True,
                     verbose    = False)
qrm_config['orderbook'] = LOB
qrm = mlob.QRM(**qrm_config)

In [ ]:
%matplotlib notebook
from IPython.display import clear_output

# Inputs : 
nb_hours_of_trading = 10

# Scheduler
sched = BackgroundScheduler()

# Live plot init
liveplot = mlob.utils.LivePlotNotebook()

# Plot update
def updateplot():
    if qrm.getBestBid():    
        liveplot.update(
            ts= qrm.getLastOrderTimestamp(),
            bestask=float(qrm.getBestAsk()),
            bestbid=float(qrm.getBestBid()),
            LOBstate = qrm.getLOBState().set_index('Price').sort_index()
        )

# Simulation starter
def startSimulation(nb_hours_of_trading=1):
    qrm.start(sched, maxruns=nb_hours_of_trading*60*60, seconds=0.01) # one sec per 0.01sec
    sched.add_job(updateplot, 'interval', seconds=1, jitter=0.5,  max_instances=1)
    sched.add_job(clear_output, 'interval', seconds=1, jitter=0.5,  max_instances=1)

sched.start()
startSimulation(nb_hours_of_trading)

### Get the trading curve

In [ ]:
dt           = 5/60**2
gamma        = 5*10**-5
timesteps    = np.arange(0, 1, 1/60**2)
tradingCurve = [q_star(x, q0, T, gamma, sigma, V, nu, dt) for x in timesteps] # an order every 5 seconds

### Create a single-asset  & single-LOB trader with a book of q0 shares

In [ ]:
agentConfig = {'orderbook'  : LOB,
               'id'         : 'myself',
               'b_record'   : True}

agent       = mlob.genericAgent(**agentConfig)

### Start liquidation

#### The LOB is simulated with a ratio of 0.0001 seconds for one real market second.
#### We must then send orders at this frequency.
#### We devide the trading curve into 3600 pieces with integer quantity

In [ ]:
agent.i_curve       = 0
quantities = np.round(tradingCurve) # quantities to detain
book = deque(maxlen = None)

def Liquidate():
    # if it is the end of the hour, send a market order with remaining quantity
    if agent.executed_quantity < q0:
        if agent.i_curve > len(quantities):
            agent.send_sell_market_order(q0 - agent.executed_quantity) 
            
            #Add sell sign in plot
            ts = agent.executedtrades[agent.executedtrades.keys()[-1]]['time']
            liveplot.addSell(ts)
            
            # Stop the simulation
#             sched.pause()
        else:
            remainingQty = q0 - agent.executed_quantity
            qty = Decimal(quantities[agent.i_curve]) - remainingQty # we execute the quantity needed to fit the curve
            
            if qty!= 0:
                agent.send_sell_market_order(Decimal(qty))
                
                #Add sell sign in plot
                ts = agent.executedtrades[agent.executedtrades.keys()[-1]]['time']
                liveplot.addSell()
                
                # get execution price
                executionPrice = agent.executedtrades[agent.executedtrades.keys()[-1]]['traded_price']

                # store in the book
                book.append({'qty'               : qty,
                             'price'             : executionPrice,
                             'remainingQuantity' : q0 - agent.executed_quantity})
                agent.i_curve += 1
            else:
                book.append({'qty'               : 0,
                             'price'             : 0,
                             'remainingQuantity' : q0 - agent.executed_quantity})
                agent.i_curve += 1

In [ ]:
sched.add_job(Liquidate, 'interval', seconds=0.01,  max_instances=1)

## Wait for the hour of liquidation to finish (3600 * 0.01 = 36seconds)
## Then stop the simulation

In [ ]:
sched.pause()

In [ ]:
Decimal(quantities[agent.i_curve])

In [ ]:
agent.executed_quantity

### Executed Trading Curve

In [ ]:
%matplotlib inline

executedQuantities  = [float(entry['qty'])               for entry in book]
executedPrices      = [float(entry['price'])             for entry in book]
remainingQuantities = [float(entry['remainingQuantity']) for entry in book]

curves = pd.DataFrame({'Realized trading Curve'    : [q0] + remainingQuantities,
                       'Theoretical trading Curve' : tradingCurve[0:len(book)+1]})
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))
curves.plot(ax=ax1); ax1.set_title('whole curve')
curves.iloc[0:60,].plot(ax=ax2); ax2.set_title('first minute')
plt.show()

#### Execution prices

In [ ]:
pd.DataFrame.from_dict(agent.executedtrades).T

In [ ]:
dfTrades = pd.DataFrame.from_dict(agent.executedtrades).T
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))
dfTrades.traded_price.astype(float).plot(ax=ax1)

distTrades = dfTrades[['traded_price', 'traded_quantity']].groupby('traded_price').sum().sort_index()
distTrades.astype(float).plot.bar(ax=ax2)

#### Executed Trades

In [ ]:
executedTrades = pd.DataFrame.from_dict(agent.executedtrades).T
executedTrades